#### Jupyter R notebook by Arvid Lundervold, 2016
in the project
### "Prediction of academic achievement in adolescents from teacher reports of inattention in childhood - a methodological pattern classification study"
by _Astri J. Lundervold, Tormod Bøe, Arvid Lundervold_


## Multinomial logistic regression

See e.g. http://www.ats.ucla.edu/stat/r/dae/mlogit.htm
and ml <- read.dta("http://www.ats.ucla.edu/stat/data/hsbdemo.dta")
Multinomial logistic regression is used to model nominal outcome variables,
in which the log odds of the outcomes are modeled as a linear combination 
of the predictor variables.
See also: Agresti_Foundations_of_Linear_and_Generalized_Linear_Models_Wiley_2015.pdf (Chap.6)

To report statistical results using R, e.g. MLR see
Andy Field et al.  Discovering Statistics Using R. Sage Publishing 2012.
http://studysites.uk.sagepub.com/dsur/main.htm and R code in
http://studysites.uk.sagepub.com/dsur/study/scriptfi.htm

Using deas from http://www.r-bloggers.com/fitting-a-neural-network-in-r-neuralnet-package
https://gist.github.com/mick001/49fad7f4c6112d954aff  neuralnetR.R

The original SPSS file as provided to AJL is 'inattention_Astri_94_96_new_grades_updated.sav'
and being edited and reduced by AJL to 'inattention_Arvid_new.sav'.

In [28]:
# Import data stored in the SPSS format
# library(memisc)
# fn <- "~/Dropbox/Arvid_Inattention/data2/inattention_Arvid_new.sav"
# data <- as.data.set(spss.system.file(fn))

Previous step is performed by __data_preparation.ipynb__ computing and storing 'inattention_nomiss_snap_is_012.csv' etc. read into this program.

In [29]:
D0 <- read.csv("~/Dropbox/Arvid_Inattention/data2/inattention_nomiss_2397x12.csv")
D1 <- read.csv("~/Dropbox/Arvid_Inattention/data2/inattention_nomiss_snap_is_012.csv")
D2 <- read.csv("~/Dropbox/Arvid_Inattention/data2/inattention_nomiss_snap_is_01.csv")
D3 <- read.csv("~/Dropbox/Arvid_Inattention/data2/inattention_nomiss_snap_is_01_outcome_is_low_medium_high.csv")
D4 <- read.csv("~/Dropbox/Arvid_Inattention/data2/inattention_nomiss_snap_is_01_outcome_is_123.csv")
data <- D4

#### Use Likert scale with three levels (not collapsing to two) 

APROPOS:
 - should actually be five (https://en.wikipedia.org/wiki/Likert_scale)

When responding to a Likert questionnaire item, respondents specify their level of agreement 
or disagreement on a symmetric agree-disagree scale for a series of statements. 
Thus, the range captures the intensity of their feelings for a given item.

The format of a typical five-level Likert item, for example, could be:
 * 1 Strongly disagree
 * 2 Disagree
 * 3 Neither agree nor disagree
 * 4 Agree
 * 5 Strongly agree

Likert scaling is a bipolar scaling method, measuring either positive or negative response to a statement.


In [30]:
for(i in 1:9){
  cmd = sprintf("data$snap%d <- D1$snap%d", i, i)
  # print(cmd)
  eval(parse(text=cmd))
}

#### Check that no datapoint is missing

In [31]:
apply(data,2,function(x) sum(is.na(x)))

gender      grade      snap1      snap2      snap3      snap4      snap5 
         0          0          0          0          0          0          0 
     snap6      snap7      snap8      snap9 averBinned 
         0          0          0          0          0

#### SIDESTEP: Randomly splitting the data into a train and a test set

In [32]:
index <- sample(1:nrow(data),round(0.75*nrow(data)))
train <- data[index,]
test <- data[-index,]

###  Fit a linear regression model and test it on the test set. 
> lm.fit <- glm(averBinned~., data=train)

> summary(lm.fit)

Before running our model. We then choose the level of our outcome that we wish 
to use as our baseline and specify this in the relevel function. 

Then, we run our model using multinom.

In [33]:
library(nnet)
data$AverageMarksLevel3 <- relevel(factor(data$averBinned), ref = "2")   # ref = "high"
test <- multinom(AverageMarksLevel3 ~ 
                   gender+grade +
                   snap1+snap2+snap3+snap4+snap5+snap6+snap7+snap8+snap9,
                   data = data)

# weights:  39 (24 variable)
initial  value 2633.373656 
iter  10 value 2518.603818
iter  20 value 2505.343951
iter  30 value 2505.055564
final  value 2505.054068 
converged


In [34]:
summary(test)

Call:
multinom(formula = AverageMarksLevel3 ~ gender + grade + snap1 + 
    snap2 + snap3 + snap4 + snap5 + snap6 + snap7 + snap8 + snap9, 
    data = data)

Coefficients:
  (Intercept)     gender      grade       snap1      snap2       snap3
1 -0.85501370  0.2256407 0.16494472  0.05488026  0.6489769 -0.02383838
3  0.07223822 -0.2620312 0.06289599 -0.52841367 -0.3073022 -0.10263580
        snap4       snap5      snap6      snap7      snap8       snap9
1 -0.16714559 -0.04084157  0.1725913 -0.1636895  0.2871769  0.06381004
3 -0.06131077  0.42519252 -0.4879066 -0.3584543 -0.3136770 -0.17760409

Std. Errors:
  (Intercept)    gender      grade     snap1     snap2     snap3     snap4
1   0.2034855 0.1056061 0.06567805 0.1374610 0.1940327 0.1950978 0.2399403
3   0.1941742 0.1039694 0.06437043 0.1836781 0.2810665 0.2518535 0.3285771
      snap5     snap6     snap7     snap8     snap9
1 0.2203130 0.2038670 0.2619213 0.1275080 0.1622366
3 0.2825674 0.2984448 0.4385224 0.1499846 0.2015068

Residu

In [35]:
z <- summary(test)$coefficients/summary(test)$standard.errors
# 2-tailed z test
p <- (1 - pnorm(abs(z), 0, 1)) * 2
p

,(Intercept),gender,grade,snap1,snap2,snap3,snap4,snap5,snap6,snap7,snap8,snap9
1,0.0000264753,0.0326285049,0.0120248910,0.6897146008,0.0008237812,0.9027510341,0.4860447980,0.8529311688,0.3972250004,0.5319993641,0.0243079335,0.6940870570
3,0.70987203,0.01172641,0.32852240,0.00401672,0.27424319,0.68362472,0.85197838,0.13238911,0.10208461,0.41369189,0.03649273,0.37811214


#### Extract the coefficients from the model and exponentiate

In [36]:
c <- exp(coef(test))

You can also use predicted probabilities to help you understand the model. 
Calculate predicted probabilities for each of the outcome levels using 
the fitted function. We can start by generating the predicted probabilities 
for the observations in our dataset and viewing the first few rows

In [37]:
head(pp <- fitted(test))

,2,1,3
1,0.3558110,0.2104557,0.4337333
2,0.3734630,0.2156931,0.4108439
3,0.3558110,0.2104557,0.4337333
4,0.3558110,0.2104557,0.4337333
5,0.3558110,0.2104557,0.4337333
6,0.3558110,0.2104557,0.4337333


#### Fore easy input to LaTeX:

In [38]:
library(stargazer)
stargazer(test)
stargazer(p)
stargazer(c)


% Table created by stargazer v.5.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Fri, Jun 10, 2016 - 17:13:14
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{2}{c}{\textit{Dependent variable:}} \\ 
\cline{2-3} 
\\[-1.8ex] & 1 & 3 \\ 
\\[-1.8ex] & (1) & (2)\\ 
\hline \\[-1.8ex] 
 gender & 0.226$^{**}$ & $-$0.262$^{**}$ \\ 
  & (0.106) & (0.104) \\ 
  & & \\ 
 grade & 0.165$^{**}$ & 0.063 \\ 
  & (0.066) & (0.064) \\ 
  & & \\ 
 snap1 & 0.055 & $-$0.528$^{***}$ \\ 
  & (0.137) & (0.184) \\ 
  & & \\ 
 snap2 & 0.649$^{***}$ & $-$0.307 \\ 
  & (0.194) & (0.281) \\ 
  & & \\ 
 snap3 & $-$0.024 & $-$0.103 \\ 
  & (0.195) & (0.252) \\ 
  & & \\ 
 snap4 & $-$0.167 & $-$0.061 \\ 
  & (0.240) & (0.329) \\ 
  & & \\ 
 snap5 & $-$0.041 & 0.425 \\ 
  & (0.220) & (0.283) \\ 
  & & \\ 
 snap6 & 0.173 & $-$0.488 \\ 
  & (0.204) & (0.298) \\ 
  & & \\ 
 sn

### Using an alternative implementation of MLR - mlogit

In [39]:
library(mlogit)

We need to modify the data so that the multinomial logistic regression
function can process it. To do this, we need to expand the outcome variable
(y) much like we would for dummy coding a categorical variable for
inclusion in standard multiple regression.

In [40]:
data1 <- data
data1$averBinned <- as.factor(data1$averBinned)
summary(data1)
data2 <- mlogit.data(data1, varying=NULL, choice="averBinned", shape="wide")

     gender          grade           snap1            snap2       
 Min.   :0.000   Min.   :2.000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.000   1st Qu.:2.000   1st Qu.:0.0000   1st Qu.:0.0000  
 Median :0.000   Median :3.000   Median :0.0000   Median :0.0000  
 Mean   :0.476   Mean   :2.814   Mean   :0.1519   Mean   :0.1377  
 3rd Qu.:1.000   3rd Qu.:3.000   3rd Qu.:0.0000   3rd Qu.:0.0000  
 Max.   :1.000   Max.   :4.000   Max.   :2.0000   Max.   :2.0000  
     snap3             snap4             snap5             snap6        
 Min.   :0.00000   Min.   :0.00000   Min.   :0.00000   Min.   :0.00000  
 1st Qu.:0.00000   1st Qu.:0.00000   1st Qu.:0.00000   1st Qu.:0.00000  
 Median :0.00000   Median :0.00000   Median :0.00000   Median :0.00000  
 Mean   :0.08803   Mean   :0.08177   Mean   :0.09929   Mean   :0.09762  
 3rd Qu.:0.00000   3rd Qu.:0.00000   3rd Qu.:0.00000   3rd Qu.:0.00000  
 Max.   :2.00000   Max.   :2.00000   Max.   :2.00000   Max.   :2.00000  
     snap7          

In [41]:
head(data2)

,gender,grade,snap1,snap2,snap3,snap4,snap5,snap6,snap7,snap8,snap9,averBinned,AverageMarksLevel3,chid,alt
1.1,0,2,0,0,0,0,0,0,0,0,0,FALSE,3,1,1
1.2,0,2,0,0,0,0,0,0,0,0,0,FALSE,3,1,2
1.3,0,2,0,0,0,0,0,0,0,0,0,TRUE,3,1,3
2.1,0,2,0,0,1,0,0,0,0,0,0,TRUE,1,2,1
2.2,0,2,0,0,1,0,0,0,0,0,0,FALSE,1,2,2
2.3,0,2,0,0,1,0,0,0,0,0,0,FALSE,1,2,3


Now we can proceed with the multinomial logistic regression analysis using
the ‘mlogit’ function and the ubiquitous ‘summary’ function of the results.
Note that the reference category is specified as “high” (2, where the levels are 0,1,2).

In [42]:
model.1 <- mlogit(averBinned ~ 1 | gender+grade +
                    snap1+snap2+snap3+snap4+snap5+snap6+snap7+snap8+snap9,
                    data = data2,
                    reflevel="2")    # reflevel = "high")

In [43]:
summary(model.1)


Call:
mlogit(formula = averBinned ~ 1 | gender + grade + snap1 + snap2 + 
    snap3 + snap4 + snap5 + snap6 + snap7 + snap8 + snap9, data = data2, 
    reflevel = "2", method = "nr", print.level = 0)

Frequencies of alternatives:
      2       1       3 
0.34126 0.32499 0.33375 

nr method
5 iterations, 0h:0m:0s 
g'(-H)^-1g = 4.51E-07 
gradient close to zero 

Coefficients :
               Estimate Std. Error t-value  Pr(>|t|)    
1:(intercept) -0.855070   0.203485 -4.2021 2.644e-05 ***
3:(intercept)  0.072173   0.194175  0.3717 0.7101245    
1:gender       0.225644   0.105606  2.1367 0.0326256 *  
3:gender      -0.262022   0.103970 -2.5202 0.0117297 *  
1:grade        0.164967   0.065678  2.5118 0.0120133 *  
3:grade        0.062916   0.064371  0.9774 0.3283691    
1:snap1        0.054873   0.137460  0.3992 0.6897525    
3:snap1       -0.528499   0.183686 -2.8772 0.0040124 ** 
1:snap2        0.648952   0.194031  3.3446 0.0008241 ***
3:snap2       -0.307387   0.281080 -1.0936 0.274133

In [44]:
b <- exp(coef(model.1))
summary(b)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4253  0.7610  0.9503  0.9806  1.1010  1.9140 